### Problem 4.1

You have been provided an dataset of 99 different shapes KIMIA-99. The task
is to find the align the remaining shapes based on the orientation of the given
template shape. Along with the code, write the flowchart of the algorithm that
you will be using to implement the following task in the Jupyter Notebook itself
as a MARKDOWN.

In [80]:
from matplotlib import pyplot as plt
import numpy as np
import cv2
import math
import os
# import io
from IPython.display import display, Image


## ALgorithm

- Convert the image into binary image

- Find the edges of the image using cv2.Canny
- Find the coordinates of the edges
- Push them all into a vector
- Find the 1st PC of this matrix
- Find the eigenvector corresponding to 1st PC of the matrix
- This eigenvector depicts the general orientation of the image
- Find the angle of this orientation
- Now er can find angles of all images and rotate appropriately

### Intuition

- Since we want to find the vector which shows most variance in the black pixels, we are considering the pixels which make up the image

- We could have taken up all the black pixels, but to optimize run-time, we took just the edge pixels
- The results work fine while taking egde coordinates as well

### Functions 
- `getFirstEigenVector()` : takes an image and returns the 1st eigenvector

- `getAngle()` : takes a vector and returns the angle the vector makes with x axis
- `Plotvector()` : plots a vector
- `rotateImage()` : rotates an image by a specified angle

In [93]:
def getFirstEigenVector(image):

    # edge detection (using cv2.Canny)
    edges = cv2.Canny(image, threshold1=30, threshold2=100)

    edge_coordinates = np.column_stack(np.where(edges > 0))

    # cov matrix of the edge points
    cov_matrix = np.cov(edge_coordinates, rowvar=False)

    # finding the 1st PC using PCA
    eigen_vals, eigen_vects = np.linalg.eigh(cov_matrix)
    sorted_indices = np.argsort(eigen_vals)[::-1]
    eigen_vals = eigen_vals[sorted_indices]
    eigen_vects = eigen_vects[:, sorted_indices]

    # getting the 1st PC eigenvector
    vec = eigen_vects[:, 0]

    # returning the 1st eigenvector
    return vec


In [94]:
def getAngle(vec):
    x,y = vec
    angle = math.atan2(y,x)
    angle = math.degrees(angle)
    return angle


In [95]:
def plotVector(vec):
    x,y = vec
    fig, ax = plt.subplots()

    ax.arrow(0, 0, x, y, head_width=0.2, head_length=0.2, fc='blue', ec='blue')

    VAL = 2*max(x,y)
    ax.set_xlim(-VAL, VAL)
    ax.set_ylim(-VAL, VAL)

    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_title('2D Vector Plot')

    plt.grid()
    plt.show()

    

In [96]:
def rotateImage(img,angle):
    height,width = img.shape[:2]
    rot_mat = cv2.getRotationMatrix2D((width / 2, height / 2), angle, 1)
    rotated_image = cv2.warpAffine(img, rot_mat, (width, height))
    rotated_image = cv2.cvtColor(rotated_image, cv2.COLOR_BGR2RGB)

    # plt.imshow(rotated_image)
    # plt.show()
    return rotated_image

In [97]:
loc = './KIMIA dataset/'
dest_loc = './output_images/'
template_img = 'f15.png'

file_list = os.listdir(loc)
ext = ['.jpg', '.jpeg', '.png', '.bmp', '.gif']
image_names = [file for file in file_list if any(file.endswith(ext) for ext in ext)]

temp = cv2.imread(loc+template_img)
e_Vect = getFirstEigenVector(temp)
ang0 = getAngle(e_Vect)
    

for name in image_names:
    path = loc+name
    img = cv2.imread(path)
    vec = getFirstEigenVector(img)
    ang = getAngle(vec)
    new_image = rotateImage(img,ang0-ang)
    cv2.imwrite(dest_loc+name,new_image)


In [86]:
# delete all files in output_images
fp = './output_images/'

if os.path.exists(fp):
    files = os.listdir(fp)

    for file in files:
        file_path = os.path.join(fp, file)
        if os.path.isfile(file_path):
            os.remove(file_path)
        else:
            print(f"The folder '{fp}' does not exist.")
